In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *
import pymysql

In [2]:
spark = SparkSession.builder.appName("field_mapping_modified").config("spark.jars","C:\spark-3.5.1-bin-hadoop3\jars\mysql-connector-j-8.4.0.jar").getOrCreate()

In [3]:
connection = pymysql.connect(
        host='localhost',
        user='root',
        password='root',
        database='main_database')

In [4]:
spark = SparkSession.builder.appName("field_mapping").config("spark.jars", "C:\spark-3.5.1-bin-hadoop3\jars\mysql-connector-j-8.4.0.jar").getOrCreate()


In [5]:
url = "jdbc:mysql://localhost:3306/main_database"
properties = {
    "user": "root",
    "password": "root",
    "driver": "com.mysql.jdbc.Driver"
}

In [6]:
def sql_table_updater(index,interval_by):
    try:
        with connection.cursor() as cursor:
            try:
                exec_date = f'update `main_database`.cf_etl_table set execution_date = current_timestamp where id = {index + 1}'
                cursor.execute(exec_date)
            except Exception as e:
                print(f"Error updating execution date: {e}")
                return

            try:
                start_date = f'update `main_database`.cf_etl_table set start_date_time = date_add(start_date_time, interval {interval_by} day)'
                cursor.execute(start_date)
            except Exception as e:
                print(f"Error updating start date: {e}")
                return

            try:
                end_date = f'update `main_database`.cf_etl_table set end_date_time = date_add(end_date_time, interval {interval_by} day)'
                cursor.execute(end_date)
            except Exception as e:
                print(f"Error updating end date: {e}")
                return

            try:
                connection.commit()
            except Exception as e:
                print(f"Error committing transaction: {e}")
                connection.rollback()
    except Exception as e:
        print(f"Error in SQL updater: {e}")

In [7]:
cursor = connection.cursor()
cursor.callproc('main_database.executor_mappings_without_cursor',[1])
result = cursor.fetchone()[0]

In [8]:
result

'tnxid AS Transaction_id,Tnx_date AS Transaction_date,acc_id AS Account_id,product AS Products,Date(Tnx_date) AS Dates'

In [9]:
query = f"(SELECT {result} FROM test_database2.sample_data1') as selected_data"

In [10]:
query

"(SELECT tnxid AS Transaction_id,Tnx_date AS Transaction_date,acc_id AS Account_id,product AS Products,Date(Tnx_date) AS Dates FROM test_database2.sample_data1') as selected_data"

In [11]:
def mapping(url,table_name,properties):
    df = spark.read.jdbc(url=url, table=table_name, properties=properties)
    rows = df.collect()
    for row in rows:
        interval_by,id,location,hdfs_file_name,inc_field,database_name,table_name,start_date,end_date,partition_by,is_inc =row['interval_days'],row['id'],row['location'],row['hdfs_file_name'],row['inc_field'],row['Schema_names'],row['Table_names'],row['start_date_time'],row['end_date_time'],row['partition_by'],row['is_incremental']
        hdfs_path = f'{location}{hdfs_file_name}'
        cursor = connection.cursor()
        cursor.callproc('main_database.executor_mappings_without_cursor',[id])
        result = cursor.fetchone()[0]
        jdbc_url = f"jdbc:mysql://localhost:3306/{database_name}"
        if is_inc:
            query = f"(SELECT {result} FROM {database_name}.{table_name} WHERE {inc_field} BETWEEN '{start_date}' AND '{end_date}') as selected_data"
            dataframe = spark.read.jdbc(url=jdbc_url, table=query, properties=properties)
            dataframe.write.mode('append').parquet(hdfs_path,partitionBy=[partition_by]) 
            sql_table_updater(rows.index(row),interval_by)
        else:
            query = f"(SELECT {result} FROM {database_name}.{table_name}) as selected_data"
            dataframe = spark.read.jdbc(url=jdbc_url, table=query, properties=properties)
            dataframe.write.mode('overwrite').parquet(hdfs_path)  

In [12]:
mapping(url,'cf_etl_table',properties)

In [15]:
data = spark.read.parquet('hdfs://localhost:19000//airflow/sample_data1')

In [16]:
data.show()

+-----------------+-------------------+-------------+-----------+----------+
|Transaction_id_ni|Transaction_date_ni|Account_id_ni|Products_ni|  Dates_ni|
+-----------------+-------------------+-------------+-----------+----------+
|                1|2023-06-23 10:30:00|       ACC001|   ProductA|2023-06-23|
|                2|2023-06-24 11:00:00|       ACC002|   ProductB|2023-06-24|
|                3|2023-06-25 09:45:00|       ACC003|   ProductC|2023-06-25|
|                4|2023-06-26 12:15:00|       ACC004|   ProductD|2023-06-26|
|                5|2023-06-27 14:30:00|       ACC005|   ProductE|2023-06-27|
+-----------------+-------------------+-------------+-----------+----------+

